# Begomg

In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

In [ ]:
import os
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import KFold, StratifiedKFold
import shutil
from sklearn.model_selection import StratifiedKFold,GroupKFold

from torch.utils.data import DataLoader, Dataset
import transformers
#datasets, transformers
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer,AutoConfig
import joblib

import random
import os
import numpy as np
import torch
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [ ]:
class CFG:
    input_path = '/content/drive/MyDrive/kaggle_competition/U.S. Patent Phrase to Phrase Matching/'


In [ ]:
cpc_texts=joblib.load('/content/drive/MyDrive/kaggle_competition/U.S. Patent Phrase to Phrase Matching/cpc_texts')
test_df = pd.read_csv(f"{CFG.input_path}test.csv")
train_df = pd.read_csv(f"{CFG.input_path}train.csv")

# deberta v3
from https://www.kaggle.com/code/yasufuminakama/pppm-deberta-v3-large-baseline-w-w-b-train

In [ ]:
from IPython.display import display
class CFG:
    wandb=True
    competition='PPPM'
    _wandb_kernel='nakama'
    print_freq=100
    num_workers=4
    model="microsoft/deberta-v3-large"
    scheduler='linear' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=50
    encoder_lr=2e-5
    decoder_lr=2e-3
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=16
    fc_dropout=0.2
    target_size=1
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3]
    train=True
    model_path = '/content/drive/MyDrive/kaggle_competition/U.S. Patent Phrase to Phrase Matching/'
    


In [ ]:
def get_logger(filename='train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()


train=train_df
test=test_df

cpc_texts=joblib.load("/content/drive/MyDrive/kaggle_competition/U.S. Patent Phrase to Phrase Matching/cpc_texts")

train['context_text'] = train['context'].map(cpc_texts)
test['context_text'] = test['context'].map(cpc_texts)

train['text'] = train['anchor'] + '[SEP]' + train['target'] + '[SEP]'  + train['context_text']
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(train.head())
display(test.head())

In [ ]:
train['score_map'] = train['score'].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4})
Fold = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train['score_map'])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
CFG.tokenizer = tokenizer

In [ ]:
from tqdm import tqdm
lengths_dict = {}

lengths = []
tk0 = tqdm(cpc_texts.values(), total=len(cpc_texts))
for text in tk0:
    length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)
lengths_dict['context_text'] = lengths

for text_col in ['anchor', 'target']:
    lengths = []
    tk0 = tqdm(train[text_col].fillna("").values, total=len(train))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        lengths.append(length)
    lengths_dict[text_col] = lengths
    
CFG.max_len = max(lengths_dict['anchor']) + max(lengths_dict['target'])\
                + max(lengths_dict['context_text']) + 4 # CLS + SEP + SEP + SEP
LOGGER.info(f"max_len: {CFG.max_len}")

In [ ]:
from torch import nn

class Deberta(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        for key in inputs.keys():
          if inputs[key].dim()==3:
            inputs[key].squeeze_(1)
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

In [ ]:
from transformers import EarlyStoppingCallback
from transformers import AutoModel,AutoConfig
from transformers import Trainer
from transformers import TrainingArguments
from torch import nn as nn
from transformers.trainer_pt_utils import get_parameter_names
from transformers.optimization import get_scheduler
from transformers import get_scheduler
from transformers.trainer_pt_utils import get_parameter_names
from transformers.optimization import get_scheduler,get_linear_schedule_with_warmup,get_cosine_schedule_with_warmup
AdamW=torch.optim.AdamW

In [ ]:
def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
          'lr': encoder_lr, 'weight_decay': weight_decay},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
          'lr': encoder_lr, 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if "model" not in n],
          'lr': decoder_lr, 'weight_decay': 0.0}
    ]
    return optimizer_parameters


# ====================================================
# scheduler
# ====================================================
def get_scheduler(cfg, optimizer, num_train_steps):
    if cfg.scheduler == 'linear':
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
        )
    elif cfg.scheduler == 'cosine':
        scheduler = get_cosine_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
        )
    return scheduler



In [ ]:
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values
        self.labels = df['score'].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        label = torch.tensor(self.labels[item], dtype=torch.float)
        return {'inputs':inputs, 'labels':label}

In [ ]:
from transformers import EarlyStoppingCallback
import scipy as sp
from apex import amp
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple, Union


def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    predictions = sp.special.expit(predictions.flatten())
    #reshape(len(predictions))
    labels = labels.flatten()
    return {
        'pearson': sp.stats.pearsonr(predictions, labels)[0]
    }
from transformers import Trainer

class FGM(object):
    """
    Example
    fgm = FGM(model,epsilon=1,emb_name='word_embeddings.')
    for batch_input, batch_label in processor:
        # 正常训练
        loss = model(batch_input, batch_label)
        loss.backward() 

        # 对抗训练
        fgm.attack() 

        loss_adv = model(batch_input, batch_label)

        loss_adv.backward()
        fgm.restore()
        optimizer.step()
        model.zero_grad()
    """

    def __init__(self, model, epsilon=1.0,emb_name='word_embeddings.'):
        self.model = model
        self.epsilon = epsilon
        self.backup = {}
        self.emb_name=emb_name

    def attack(self):
        emb_name=self.emb_name
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0 and not torch.isnan(norm):
                    r_at = self.epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self):
        emb_name = self.emb_name
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}


class Trainers(Trainer):

  def compute_loss(self, model, inputs, return_outputs=False): ## compute loss这个步骤实际上定义了 model的forward和output以及model的loss计算的结果
      labels = inputs.get("labels")
      logits = model(inputs.get('inputs')) ##在这里定义了foward和batch的计算过程
      loss_fct = nn.BCEWithLogitsLoss(reduction="mean") ## loss可以不断重新定义无所谓的
      loss = loss_fct(logits.view(-1, 1), labels.float().view(-1,1))
      #final_loss = torch.masked_select(loss, labels.view(-1, 1) != -1).mean()
      #return (torch.masked_select(loss, labels.view(-1, 1) != -1).mean(), outputs) if return_outputs else loss
      return (loss, {'outputs':logits}) if return_outputs else loss

    # We can add any adversial training methods here by custom training step of huggingface trainer
  def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        model.train()
        inputs = self._prepare_inputs(inputs)
        fgm = FGM(model,epsilon=1,emb_name='word_embeddings.')

        with self.autocast_smart_context_manager():
            loss = self.compute_loss(model, inputs)
            loss = loss / self.args.gradient_accumulation_steps
            loss=self.scaler.scale(loss) ##半精度用torch自带的就行了
            loss.backward()        
            fgm.attack() # 在embedding上添加对抗扰动
            loss_adv = self.compute_loss(model, inputs)
            loss_adv.backward()
            fgm.restore()
        return loss_adv.detach()

    
es=EarlyStoppingCallback(early_stopping_patience=10)


args=TrainingArguments(output_dir='/content/drive/tmp/deberta_temp',evaluation_strategy='steps',prediction_loss_only =False,overwrite_output_dir =True, \
                       per_device_train_batch_size=16,per_device_eval_batch_size=256,learning_rate=2e-5, \
                       weight_decay=0.01,max_grad_norm=1,num_train_epochs =50,logging_strategy ='epoch',save_steps=72*2,save_total_limit =1,seed=42,eval_steps=72*2,dataloader_num_workers=4, \
                       fp16=True,dataloader_drop_last =False,disable_tqdm =False,load_best_model_at_end=True,sharded_ddp=False,label_smoothing_factor=0.000, \
                       group_by_length=False,gradient_accumulation_steps=1*16,dataloader_pin_memory=False, \
                       metric_for_best_model='pearson', greater_is_better=True,) 


In [ ]:
import gc
y_oof=np.zeros(train_df.shape[0])
pearsons=0
model_function=Deberta
datasets=TrainDataset
args=args
callbacks=[es]


for i in range(5):
  model=model_function(CFG,config_path=None, pretrained=True)
  if i==0:
    torch.save(model.state_dict(), CFG.model_path+CFG.model.split('/')[1]+'-'+'initialization')

  model.load_state_dict(torch.load(CFG.model_path+CFG.model.split('/')[1]+'-'+'initialization'))

  model.train()

  train_dataset=datasets(CFG,train.query('fold!=@i'))

  eval_dataset=datasets(CFG,train.query('fold==@i'))

  optimizer_parameters = get_optimizer_params(model,
                                            encoder_lr=CFG.encoder_lr, 
                                            decoder_lr=CFG.decoder_lr,
                                            weight_decay=CFG.weight_decay)
  AdamW=torch.optim.AdamW
  optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
  num_train_steps = int(train_df.query('fold!=@i').size / CFG.batch_size * 5)
  lr_scheduler = get_scheduler(CFG, optimizer, num_train_steps)


  trainer = Trainers(
      model,
      args,
      train_dataset = train_dataset,
      eval_dataset = eval_dataset,
      tokenizer=None,
      data_collator = None,
      callbacks=callbacks,
      optimizers=(optimizer,lr_scheduler),compute_metrics=compute_metrics
  )
  trainer.data_collator=None#You must set the data_collator=None

  trainer.train()

  torch.save(trainer.model.state_dict(), CFG.model_path+CFG.model.split('/')[1]+'-best_fold_{}'.format(i))
  y_oof[train_df.query('fold==@i').index]=trainer.predict(eval_dataset).predictions.flatten()
  pearsons+=(compute_metrics((y_oof[train_df.query('fold==@i').index],train_df.query('fold==@i').score.values))).get('pearson')/5.0
  del model,trainer,train_dataset,eval_dataset;
  gc.collect()
  torch.cuda.empty_cache()

oof_pearson=compute_metrics((y_oof,train_df.score.values)).get('pearson')
print(f'average pearson is {pearsons}')
print(f'oof pearson is {oof_pearson}')
#pearsons,oof_pearson